In [1]:
from google.cloud import bigquery
project_id="students-group2"
client = bigquery.Client(project=project_id)

In [2]:
# Lister toutes les tables dans le dataset master
dataset_id = "master-ai-cloud.MoviePlatform"
tables = list(client.list_tables(dataset_id))

for table in tables:
    print(table.table_id)

movies
ratings


In [12]:
# DataFrames depuis le dataset principal
df_movies = client.query("""SELECT * FROM `master-ai-cloud.MoviePlatform.movies`""").to_dataframe()
df_ratings = client.query("""SELECT * FROM `master-ai-cloud.MoviePlatform.ratings`""").to_dataframe()

In [13]:
from google.cloud import bigquery

dataset_secondary_id = "students-group2.fatma_aziz"
dataset = bigquery.Dataset(dataset_secondary_id)
dataset.location = "US"

client.create_dataset(dataset, exists_ok=True)

print("Dataset secondaire prêt :", dataset_secondary_id)

Dataset secondaire prêt : students-group2.fatma_aziz


In [15]:
df_movies.to_gbq(
    destination_table="students-group2.fatma_aziz.movies",
    project_id="students-group2",
    if_exists="replace" 
)
print("Table movies copiée avec succès")

100%|██████████| 1/1 [00:00<00:00, 7584.64it/s]

Table movies copiée avec succès


In [16]:
df_ratings.to_gbq(
    destination_table="students-group2.fatma_aziz.ratings",
    project_id="students-group2",
    if_exists="replace"
)
print("Table ratings copiée avec succès")

100%|██████████| 1/1 [00:00<00:00, 4614.20it/s]

Table ratings copiée avec succès


In [17]:
# Dataframes 
df_movies = client.query("SELECT * FROM `students-group2.fatma_aziz.movies`").to_dataframe()
df_ratings = client.query("SELECT * FROM `students-group2.fatma_aziz.ratings`").to_dataframe()

In [18]:
print("Doublons dans ratings :", df_ratings.duplicated().sum())
print("Doublons dans movies :", df_movies.duplicated().sum())

Doublons dans ratings : 0
Doublons dans movies : 0


In [19]:
df_movies['year'] = df_movies['title'].str.extract(r'\((\d{4})\)')[0].astype('Int64')

In [20]:
df_movies['genres_list'] = df_movies['genres'].str.split('|')

In [21]:
df_movies

,movieId,title,genres,year,genres_list
0,126929,Li'l Quinquin ( ),(no genres listed),<NA>,[(no genres listed)]
1,135460,Pablo (2012),(no genres listed),2012,[(no genres listed)]
2,138863,The Big Broadcast of 1936 (1935),(no genres listed),1935,[(no genres listed)]
3,141305,Round Trip to Heaven (1992),(no genres listed),1992,[(no genres listed)]
4,141472,The 50 Year Argument (2014),(no genres listed),2014,[(no genres listed)]
...,...,...,...,...,...
10324,85896,Tribute to a Bad Man (1956),Western,1956,[Western]
10325,103570,Dead Man's Burden (2012),Western,2012,[Western]
10326,105223,Colorado Territory (1949),Western,1949,[Western]
10327,128360,The Hateful Eight (2015),Western,2015,[Western]


In [22]:
df_ratings

,userId,movieId,rating,timestamp
0,1,204,0.5,1217895786
1,1,256,0.5,1217895764
2,1,277,0.5,1217895772
3,1,719,0.5,1217895799
4,1,45950,0.5,1217897813
...,...,...,...,...
105334,668,93040,5.0,1331051757
105335,668,98154,5.0,1353208964
105336,668,101862,5.0,1373168467
105337,668,106916,5.0,1388740601


In [23]:
from sklearn.preprocessing import LabelEncoder

user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()

df_ratings['user_idx'] = user_encoder.fit_transform(df_ratings['userId'])
df_ratings['movie_idx'] = movie_encoder.fit_transform(df_ratings['movieId'])

In [24]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df_ratings, test_size=0.2, random_state=42)

In [25]:
import os

# Chemin relatif vers la racine du projet
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))  # remonte d'un dossier
data_path = os.path.join(project_root, 'data')

# Créer le dossier data à la racine 
os.makedirs(data_path, exist_ok=True)

print("Le dossier data sera créé ici :", data_path)



Le dossier data sera créé ici : /home/jupyter/Fatma_Aziz/movie_recommender/data


In [26]:
train_df.to_csv(os.path.join(data_path, 'train_ratings.csv'), index=False)
test_df.to_csv(os.path.join(data_path, 'test_ratings.csv'), index=False)
df_movies.to_csv(os.path.join(data_path, 'movies.csv'), index=False)

In [27]:
import pickle
# Sauvegarder les encoders
with open(os.path.join(data_path, 'user_encoder.pkl'), 'wb') as f:
    pickle.dump(user_encoder, f)

with open(os.path.join(data_path, 'movie_encoder.pkl'), 'wb') as f:
    pickle.dump(movie_encoder, f)

print("✅ Encoders sauvegardés dans", data_path)

✅ Encoders sauvegardés dans /home/jupyter/Fatma_Aziz/movie_recommender/data
